<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/%ED%9E%88%EC%8A%A4%ED%86%A0%EA%B7%B8%EB%9E%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

#가격대 별로 나눈 히스토그램

In [3]:
%%bigquery --project ringed-marker-347607 dt
WITH
stats AS (
    SELECT
    -- 금액의 최댓값 + 1
    MAX(price) + 1 AS max_price
    -- 금액의 최솟값
    , MIN(price) AS min_price
    -- 금액의 범위 + 1(실수)
    , MAX(price) + 1 - MIN(price) AS range_price
    -- 계층 수
    , 10 AS bucket_num
  FROM
    ch4.purchase_detail_log
)
, purchase_log_with_bucket AS (
  SELECT
    price
    , min_price
    -- 정규화 금액: 대상 금액 - 최소 금액
    , price - min_price AS diff
    -- 계층 범위 : 금액 범위를 계층 수로 분할
    , 1.0 * range_price / bucket_num AS bucket_range

    -- 계층 판정 : FLOOR(정규화 금액 / 계층 범위)
    , FLOOR(
        1.0 * (price - min_price) / (1.0 * range_price / bucket_num)
        -- index가 1부터 시작하므로, 1을 더함
    ) + 1 AS bucket

  FROM
    ch4.purchase_detail_log, stats
)
SELECT
  bucket
  -- 계층의 하한과 상한 계산
  , min_price + bucket_range * (bucket - 1) AS lower_limit
  , min_price + bucket_range * bucket AS upper_limit
  -- 도수 세기
  , COUNT(price) AS num_purchase
  -- 합계 금액 계산
  , SUM(price) AS total_amount
FROM
  purchase_log_with_bucket
GROUP BY
  bucket, min_price, bucket_range
ORDER BY bucket
;

# 임의의 가격대로 나눈 히스토그램

In [4]:
%%bigquery --project ringed-marker-347607 dt2
WITH
stats AS (
    SELECT
    -- 금액의 최대값
    50000 AS max_price
    -- 금액의 최솟값
    , 0 AS min_price
    -- 금액의 범위
    , 50000 AS range_price
    -- 계층 수
    , 10 AS bucket_num
  FROM
    ch4.purchase_detail_log
)
, purchase_log_with_bucket AS (
  SELECT
    price
    , min_price
    -- 정규화 금액: 대상 금액 - 최소 금액
    , price - min_price AS diff
    -- 계층 범위 : 금액 범위를 계층 수로 분할 ----->간격이 5000단위로 깔끔함
    , 1.0 * range_price / bucket_num AS bucket_range

    -- 계층 판정 : FLOOR(정규화 금액 / 계층 범위)
    , FLOOR(
        1.0 * (price - min_price) / (1.0 * range_price / bucket_num)
        -- index가 1부터 시작하므로, 1을 더함
    ) + 1 AS bucket

  FROM
    ch4.purchase_detail_log, stats
)
SELECT
  bucket
  -- 계층의 하한과 상한 계산
  , min_price + bucket_range * (bucket - 1) AS lower_limit
  , min_price + bucket_range * bucket AS upper_limit
  -- 도수 세기
  , COUNT(price) AS num_purchase
  -- 합계 금액 계산
  , SUM(price) AS total_amount
FROM
  purchase_log_with_bucket
GROUP BY
  bucket, min_price, bucket_range
ORDER BY bucket
;